# **Importing Libraries**

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import sklearn

: 

In [2]:
#Give dataset path
train_path = 'train'
test_path = 'test'

In [3]:
folders = glob('D:\\Projects_CNN\\Chest Cancer\\chest-ctscan-images\\Data\\train')
len(folders)

1

In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# **Designing Our Model**

In [5]:
InceptionV3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

# The last 15 layers fine tune
# for layer in InceptionV3_model.layers[:-15]:
#     layer.trainable = False

x = InceptionV3_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)
output  = Dense(units=4, activation='softmax')(x)
model = Model(InceptionV3_model.input, output)


model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [7]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])

# **Image Augmentation**

In [8]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
#no flip and zoom for test datase

In [9]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('D:\\Projects_CNN\\Chest Cancer\\chest-ctscan-images\\Data\\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 613 images belonging to 4 classes.


In [10]:
test_set = test_datagen.flow_from_directory('D:\\Projects_CNN\\Chest Cancer\\chest-ctscan-images\\Data\\test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 315 images belonging to 4 classes.


# **Training Our Model**

In [11]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\AJAY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
20/20 [==============================] - 418s 19s/step - loss: 1.1345 - accuracy: 0.5073 - val_loss: 31255.3203 - val_accuracy: 0.1714
Epoch 2/20
20/20 [==============================] - 454s 23s/step - loss: 1.0606 - accuracy: 0.6085 - val_loss: 312487.4062 - val_accuracy: 0.1714
Epoch 3/20
10/20 [==============>...............] - ETA: 3:24 - loss: 0.8780 - accuracy: 0.6416

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# predict probabilities for test set
yhat_probs = model.predict(test_set, verbose=0)
yhat_probs

# Precision , Recall, F1-Score, Support, Accuracies

In [ ]:
Y_pred = model.predict(test_set, test_set.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_set.classes
from sklearn.metrics import classification_report
report = classification_report(val_trues, val_preds)
print(report)
cm = metrics.confusion_matrix(val_trues, val_preds)
cm

In [ ]:
recall = sklearn.metrics.recall_score(val_trues, val_preds, average='micro', pos_label="positive")
print(recall)

In [ ]:
val_trues

# Consusion Matrix

In [ ]:
# Print the confusion matrix using Matplotlib

fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(cm, cmap=plt.cm.Oranges, alpha=0.3)

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(x=j, y=i,s=cm[i, j], va='center', ha='center', size='xx-large')

plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model.save('modelcancerlung.h5')

# **Making Predictions**

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
classes = ["AdenocarcinomaChest Lung Cancer ","Large cell carcinoma Lung Cancer" , "NO Lung Cancer/ NORMAL" , "Squamous cell carcinoma Lung Cancer"]

In [ ]:
def predict_image(img):
    plt.figure(figsize=(40,8))
    print()
    print('-----------------------------------Chest Cancer Type Detection---------------------------------------------------')
    print()
    print('----------------------------------------------RESULT-------------------------------------------------------------')
    print()
    x=image.img_to_array(img)
    x=x/255
    plt.imshow(img)
    x=np.expand_dims(x,axis=0)
    #img_data=preprocess_input(x)

    print(classes[np.argmax(model.predict(x))])

In [ ]:
im1=image.load_img('D:\\Chest Cancer\\chest-ctscan-images\\Data\\train\\adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib\\000103 (9).png',target_size=(224,224))
predict_image(im1)

In [ ]:
im2=image.load_img('D:\\Chest Cancer\\chest-ctscan-images\\Data\\test\\adenocarcinoma\\000114.png',target_size=(224,224))
predict_image(im2)

In [ ]:
im3=image.load_img('D:\\Chest Cancer\\chest-ctscan-images\\Data\\test\\adenocarcinoma\\000118 (7).png',target_size=(224,224))
predict_image(im3)

In [ ]:
im4=image.load_img('D:\\Chest Cancer\\chest-ctscan-images\\Data\\test\\adenocarcinoma\\000119 (4).png',target_size=(224,224))
predict_image(im4)

In [ ]:
im5=image.load_img('D:\\Chest Cancer\\chest-ctscan-images\\Data\\test\\adenocarcinoma\\000113 (7).png',target_size=(224,224))
predict_image(im5)